In [ ]:
import cv2
import numpy as np
from tensorflow.keras.models import load_model

# Constants
IMAGE_SIZE = 128
MODEL_PATH = 'fingers_detection.keras'

# Load trained model
model = load_model(MODEL_PATH)

# Define ROI dimensions
ROI_SIZE = 170  # You can adjust as needed
FRAME_WIDTH = 640
FRAME_HEIGHT = 480
ROI_TOP_LEFT = ((FRAME_WIDTH - ROI_SIZE) // 2, (FRAME_HEIGHT - ROI_SIZE) // 2)

def preprocess_frame(roi):
    gray = cv2.cvtColor(roi, cv2.COLOR_BGR2GRAY)
    gray = cv2.resize(gray, (IMAGE_SIZE, IMAGE_SIZE))
    gray = gray.astype(np.float32) / 255.0
    gray = np.reshape(gray, (1, IMAGE_SIZE, IMAGE_SIZE, 1))
    return gray

# Start webcam
cap = cv2.VideoCapture(0)
cap.set(cv2.CAP_PROP_FRAME_WIDTH, FRAME_WIDTH)
cap.set(cv2.CAP_PROP_FRAME_HEIGHT, FRAME_HEIGHT)

while True:
    ret, frame = cap.read()
    if not ret:
        break

    frame = cv2.flip(frame, 1)

    # Draw ROI rectangle
    x, y = ROI_TOP_LEFT
    roi = frame[y:y+ROI_SIZE, x:x+ROI_SIZE]
    cv2.rectangle(frame, (x, y), (x+ROI_SIZE, y+ROI_SIZE), (0, 255, 0), 2)

    # Preprocess ROI and predict
    if roi.shape[0] == ROI_SIZE and roi.shape[1] == ROI_SIZE:
        processed = preprocess_frame(roi)
        prediction = model.predict(processed, verbose=0)
        label = "Finger 1" if prediction[0][0] >= 0.5 else "Other"
        cv2.putText(frame, label, (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)

    cv2.imshow("Webcam Prediction", frame)

    # Exit on 'q'
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()


In [ ]:
import cv2
import numpy as np
import tensorflow as tf
import os

# Constants
IMAGE_SIZE = 128
MODEL_PATH = 'fingers_detection.keras'
IMAGE_DIR = 'fingers/test'

# Load model
model = tf.keras.models.load_model(MODEL_PATH)

class FingerClassifier:
    def __init__(self, model_object):
        self.detect = model_object

    def get_classification(self, img):
        img = img.reshape(1, IMAGE_SIZE, IMAGE_SIZE, 1)  # shape (1, 128, 128, 1)
        img = tf.constant(img, dtype=float)
        output = self.detect(img)
        pred = output.numpy()[0][0]
        return 1 if pred > 0.5 else 0

def process_image(img):
    img = cv2.resize(img, (IMAGE_SIZE, IMAGE_SIZE))
    img = img.astype(np.float32) / 255.0
    img = np.reshape(img, (IMAGE_SIZE, IMAGE_SIZE, 1))
    return img

# Initialize classifier
classifier = FingerClassifier(model)

# List image files
image_files = [f for f in os.listdir(IMAGE_DIR) if f.lower().endswith(('.png', '.jpg', '.jpeg'))]

if not image_files:
    print(f"No images found in '{IMAGE_DIR}'")
else:
    for file_name in image_files:
        file_path = os.path.join(IMAGE_DIR, file_name)
        img = cv2.imread(file_path)

        if img is None:
            print(f"Could not read: {file_name}")
            continue

        gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

        # Crop from center
        h, w = gray.shape
        crop_size = 250
        start_x = max((w - crop_size) // 2, 0)
        start_y = max((h - crop_size) // 2, 0)
        cropped = gray[start_y:start_y + crop_size, start_x:start_x + crop_size]

        # Process and classify
        processed = process_image(cropped)
        prediction = classifier.get_classification(processed)
        label = "1 Finger" if prediction == 1 else "Other"

        # Prepare display image
        display_img = (processed[:, :, 0] * 255).astype(np.uint8)
        cv2.putText(display_img, label, (10, 25), cv2.FONT_HERSHEY_SIMPLEX, 0.8, 255, 2)

        # Show only processed image with label
        cv2.imshow("Processed with Prediction", display_img)

        key = cv2.waitKey(0)
        if key == 27:  # ESC key to exit early
            break

cv2.destroyAllWindows()


In [ ]:
# to stop camera
cv2.destroyAllWindows()